In [1]:
%pip install langchain-chroma langchain-community langchain-openai langchain-text-splitters pandas python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma

In [3]:
from dotenv import load_dotenv

load_dotenv()

False

In [4]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [5]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [6]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [7]:
books["tagged_description"].to_csv(
    "tagged_description.txt",
    index=False,
    header=False,
    encoding="utf-8"
)

In [8]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

# 1. Thêm encoding="utf-8" để tránh lỗi đọc file trên Windows
loader = TextLoader("tagged_description.txt", encoding="utf-8") 
raw_documents = loader.load()

# 2. Sửa chunk_size > 0. 
# Nếu mục đích của bạn là "mỗi dòng là một document riêng biệt", 
# hãy đặt chunk_size lớn hơn độ dài dòng dài nhất trong file của bạn.
text_splitter = CharacterTextSplitter(
    separator="\n",    # Cắt dựa trên dấu xuống dòng
    chunk_size=2000,   # Đủ lớn để chứa trọn vẹn 1 dòng (description) mà không bị cắt đôi
    chunk_overlap=0
)

documents = text_splitter.split_documents(raw_documents)

# Kiểm tra kết quả
print(f"Số lượng documents tạo ra: {len(documents)}")
print(f"Nội dung document đầu tiên: {documents[0].page_content}")

Created a chunk of size 2010, which is longer than the specified 2000
Created a chunk of size 2012, which is longer than the specified 2000
Created a chunk of size 2834, which is longer than the specified 2000
Created a chunk of size 2510, which is longer than the specified 2000
Created a chunk of size 2008, which is longer than the specified 2000
Created a chunk of size 2285, which is longer than the specified 2000
Created a chunk of size 2616, which is longer than the specified 2000
Created a chunk of size 2032, which is longer than the specified 2000
Created a chunk of size 2762, which is longer than the specified 2000
Created a chunk of size 2005, which is longer than the specified 2000
Created a chunk of size 2141, which is longer than the specified 2000
Created a chunk of size 2209, which is longer than the specified 2000
Created a chunk of size 2877, which is longer than the specified 2000
Created a chunk of size 2400, which is longer than the specified 2000
Created a chunk of s

Số lượng documents tạo ra: 1588
Nội dung document đầu tiên: "9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a

In [9]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, G

In [10]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
# SỬA Ở ĐÂY: Dùng tqdm thường, không dùng tqdm.notebook
from tqdm import tqdm 

# 1. Khởi tạo model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 2. Khởi tạo Database
db_books = Chroma(
    collection_name="books_collection",
    embedding_function=embedding_model
)

# 3. Kiểm tra lại số lượng documents
total_docs = len(documents)
print(f"Tổng số văn bản cần xử lý: {total_docs}")

# Nếu số lượng quá lớn (>50.000), bạn nên giảm batch_size xuống 32
batch_size = 64 

print("Bắt đầu xử lý...")

# Vòng lặp nạp dữ liệu
for i in tqdm(range(0, total_docs, batch_size), desc="Đang tạo Vector DB"):
    batch = documents[i : i + batch_size]
    db_books.add_documents(batch)

print("Đã tạo Vector DB thành công!")

Tổng số văn bản cần xử lý: 1588
Bắt đầu xử lý...


Đang tạo Vector DB: 100%|██████████| 25/25 [00:43<00:00,  1.73s/it]

Đã tạo Vector DB thành công!


In [11]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(id='666bf1e8-1466-475c-b053-e05cda541001', metadata={'source': 'tagged_description.txt'}, page_content='"9780064404419 ""Culled from 69 stories collected in a [1930s] WPA project, [these 20] tales are organized into sections with themes like \'Tricksters\' or \'Virtues and Vices,\' each with a thoughtful introduction placing the individual stories in the context of feelings and background of the original tellers. Yep\'s telling is vigorous, often poetic, imbued with earthy humor and realism touched with fatalism. A handsomely designed collection."" —K. Notable Children\'s Books of 1989 (ALA) The USA Through Children\'s Books 1990 (ALA) 1989 Boston Globe–Horn Book Award for Nonfiction 1990 Fanfare Honor List (The Horn Book) 1989 Children\'s Editors\' Choices (BL) Notable 1989 Children\'s Trade Books in Social Studies (NCSS/CBC) Children\'s Books of 1989 (Library of Congress) 1989 Children\'s Books (NY Public Library) ""The Best Books"" 1989 (Parents Magazine)"\n"9780064404891 

In [12]:
import pandas as pd

def retrieve_semantic_recommendations(
    query: str,
    top_k: int = 10,
) -> pd.DataFrame:
    
    # 1. Tìm kiếm nhiều hơn top_k một chút để trừ hao các trường hợp lỗi ID
    recs = db_books.similarity_search(query, k=top_k * 2)
    
    books_list = []
    
    for rec in recs:
        # Lấy nội dung văn bản
        content = rec.page_content.strip('"').strip()
        
        # 2. Dùng Try-Except để tránh crash nếu dòng đó không bắt đầu bằng số
        try:
            # Giả định từ đầu tiên là ISBN
            book_id = int(content.split()[0])
            
            # Chỉ thêm vào danh sách nếu chưa có (tránh trùng lặp)
            if book_id not in books_list:
                books_list.append(book_id)
                
        except (ValueError, IndexError):
            # Nếu dòng này bị lỗi format, bỏ qua và đi tiếp
            continue
            
        # Dừng lại khi đã đủ số lượng yêu cầu
        if len(books_list) >= top_k:
            break

    # 3. Lấy thông tin sách nhưng GIỮ NGUYÊN THỨ TỰ GỢI Ý
    # Lọc ra các sách có trong danh sách tìm được
    results = books[books["isbn13"].isin(books_list)]
    
    # Sắp xếp lại kết quả theo đúng thứ tự của books_list (độ phù hợp giảm dần)
    # Tạo cột tạm để sort
    results = results.set_index("isbn13")
    results = results.reindex(books_list).reset_index()
    
    return results

# --- Test thử hàm ---
# Hãy thử tìm một nội dung cụ thể
results = retrieve_semantic_recommendations("a story about magic and wizards", top_k=5)

# Hiển thị kết quả (chỉ hiện tên sách và mô tả để dễ nhìn)
# Giả sử cột tên sách là 'title', bạn hãy đổi tên cột cho đúng file của bạn
if not results.empty:
    display(results[['isbn13', 'title', 'authors']].head())
else:
    print("Không tìm thấy kết quả nào hoặc lỗi định dạng dữ liệu.")

,isbn13,title,authors
0,9780441014439,Harrowing the Dragon,Patricia A. McKillip
1,9781587150883,The Well at the World's End,William Morris;Lin Carter
2,9780374525866,The Magic Barrel,Bernard Malamud
3,9780694524754,The Chronicles of Narnia CD Box Set,C. S. Lewis
4,9780439682589,Harry Potter,J. K. Rowling


In [13]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]

In [14]:
retrieve_semantic_recommendations("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
59,9780007151240,0007151241,The Family Way,Tony Parsons,Parenthood,http://books.google.com/books/content?id=dJEIx...,It should be the most natural thing in the wor...,2005.0,3.51,400.0,2095.0,The Family Way,9780007151240 It should be the most natural th...
104,9780060245603,0060245603,Today I Feel Silly & Other Moods That Make My Day,Jamie Lee Curtis,Juvenile Fiction,http://books.google.com/books/content?id=s_2GI...,Today I feel silly. Mom says it's the heat. I ...,1998.0,4.15,40.0,4030.0,Today I Feel Silly & Other Moods That Make My Day,9780060245603 Today I feel silly. Mom says it'...
186,9780060622138,006062213X,Shadow of the Almighty,Elisabeth Elliot,Religion,http://books.google.com/books/content?id=QQCG9...,The bestselling account of the martyrdom of Ji...,1989.0,4.39,272.0,6658.0,Shadow of the Almighty: The Life and Testament...,9780060622138 The bestselling account of the m...
270,9780060885373,0060885378,Little House in the Big Woods,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=7JctN...,A year in the life of two young girls growing ...,2007.0,4.18,198.0,193862.0,Little House in the Big Woods,9780060885373 A year in the life of two young ...
328,9780060974992,0060974990,Savage Inequalities,Jonathan Kozol,Education,http://books.google.com/books/content?id=UEJ3Q...,National Book Award-winning author Jonathan Ko...,1992.0,4.24,272.0,15021.0,Savage Inequalities: Children in America's Sch...,9780060974992 National Book Award-winning auth...
391,9780061205699,0061205699,To Kill a Mockingbird (slipcased edition),Harper Lee,Fiction,http://books.google.com/books/content?id=M9lKH...,"At the age of eight, Scout Finch is an entrenc...",2006.0,4.27,323.0,250.0,To Kill a Mockingbird (slipcased edition),"9780061205699 At the age of eight, Scout Finch..."
404,9780064402453,0064402452,Racso and the Rats of NIMH,Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=MgoNv...,"‘Racso, a brash and boastful little rodent, is...",1988.0,3.76,288.0,3231.0,Racso and the Rats of NIMH,"9780064402453 ‘Racso, a brash and boastful lit..."
407,9780064404419,0064404412,The Rainbow People,Laurence Yep,Juvenile Fiction,http://books.google.com/books/content?id=5AHwq...,"""Culled from 69 stories collected in a [1930s]...",1992.0,3.75,208.0,202.0,The Rainbow People,"9780064404419 ""Culled from 69 stories collecte..."
411,9780064405478,0064405478,Betsy Was a Junior,Maud Hart Lovelace,Juvenile Fiction,http://books.google.com/books/content?id=TNcQQ...,A small town shortly after the turn of the cen...,1995.0,4.17,320.0,4744.0,Betsy Was a Junior,9780064405478 A small town shortly after the t...
415,9780064406307,006440630X,The Midwife's Apprentice (rpkg),Karen Cushman,Juvenile Fiction,http://books.google.com/books/content?id=Bhm76...,"'Like Cushman's 1995 Newbery Honor Book, Cathe...",1996.0,3.72,128.0,35319.0,The Midwife's Apprentice (rpkg),9780064406307 'Like Cushman's 1995 Newbery Hon...
